In [2]:
import pandas as pd
import os

In [3]:
bene_files = os.listdir('files')

In [34]:
files = {}
for file_name in bene_files:
    file_ext = file_name.split('.')[-1:][0]
    if (file_ext == 'xlsx' and 'Beneficiary_Summary_File' in file_name):
       file_year = file_name.split('_')[2]
       df = pd.read_excel(os.path.join('files', file_name), 'Sheet1')
       df['file_year'] = file_year
       files[file_name] = df

In [52]:
df = pd.concat(files.values())

In [53]:
diabetes = df['SP_DIABETES'].map({1: 'Diabetes', 2: 'No Diabetes'})

In [54]:
chf = df['SP_CHF'].map({1: 'Congestive Heart Failure', 2: 'Healthy heart'})

In [55]:
df['diabetes'] = diabetes

In [56]:
df['chf'] = chf

In [57]:
df['total_payments'] = df['MEDREIMB_IP'] + df['MEDREIMB_OP'] + df['MEDREIMB_CAR']

In [58]:
df.dtypes

DESYNPUF_ID                  object
BENE_BIRTH_DT                 int64
BENE_DEATH_DT               float64
BENE_SEX_IDENT_CD             int64
BENE_RACE_CD                  int64
BENE_ESRD_IND                object
SP_STATE_CODE                 int64
BENE_COUNTY_CD                int64
BENE_HI_CVRAGE_TOT_MONS       int64
BENE_SMI_CVRAGE_TOT_MONS      int64
BENE_HMO_CVRAGE_TOT_MONS      int64
PLAN_CVRG_MOS_NUM             int64
SP_ALZHDMTA                   int64
SP_CHF                        int64
SP_CHRNKIDN                   int64
SP_CNCR                       int64
SP_COPD                       int64
SP_DEPRESSN                   int64
SP_DIABETES                   int64
SP_ISCHMCHT                   int64
SP_OSTEOPRS                   int64
SP_RA_OA                      int64
SP_STRKETIA                   int64
MEDREIMB_IP                   int64
BENRES_IP                     int64
PPPYMT_IP                     int64
MEDREIMB_OP                   int64
BENRES_OP                   

In [59]:
pvt = df.pivot_table(index=['file_year', 'diabetes', 'chf'], values=['BENE_HI_CVRAGE_TOT_MONS', 'total_payments'], aggfunc='sum')

In [60]:
pvt  = pvt.reset_index()

In [61]:
def calc_pmpy(x):
    member_months = x['BENE_HI_CVRAGE_TOT_MONS']
    amount = x['total_payments']
    if member_months > 0:
       return amount/member_months
    else:
       return 0

In [62]:
pvt['PMPY'] = pvt.apply(calc_pmpy, axis=1)

In [65]:
pvt.pivot_table(index=['diabetes', 'chf'], columns='file_year', values='PMPY', aggfunc='mean')

file_year                                    2008        2009        2010
diabetes    chf                                                          
Diabetes    Congestive Heart Failure  1136.899239  917.351984  514.541373
            Healthy heart              354.527616  450.920139  295.011628
No Diabetes Congestive Heart Failure   403.659118  459.018555  344.250559
            Healthy heart               68.192924  103.178112   91.186798

In [66]:
pvt.to_excel('pvt_results.xlsx')